In [0]:
import os
import tweepy
import pandas as pd

References: 
<br />
https://towardsdatascience.com/how-to-access-twitters-api-using-tweepy-5a13a206683b

In [0]:
import getpass # prevents echo that is unhidden

# allow users to key in tokens here:
consumerkey = getpass.getpass("Consumer Key: ")
consumersecret = getpass.getpass("Consumer Secret: ")
accesstoken = getpass.getpass("Access Token: ")
accesstokensecret = getpass.getpass("Access Token Secret: ")

Consumer Key: ··········
Consumer Secret: ··········
Access Token: ··········
Access Token Secret: ··········


In [0]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumerkey, consumersecret)
auth.set_access_token(accesstoken,accesstokensecret)

api = tweepy.API(auth)

# Singapore Airlines

In [0]:
userid = 'SingaporeAir'

In [0]:
# finding out about Singapore Air
sgair_info = api.get_user(screen_name=userid)

# this returns a User object, params see here: https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object
# for Tweet objects, https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object 

In [0]:
sgair_info.description

'Tell us your travel stories with #FlySQ #SingaporeAir. For feedback, write to https://t.co/kvBYaqY53O. For guidelines on our social media channels, refer: https://t.co/QHPadqn54r'

In [0]:
# getting Singapore Airlines' 200 most recent posts
timeline = api.user_timeline(screen_name=userid, count=500)

In [0]:
type(timeline)
# tweepy's ResultSet is a list like object that holds results from a Twitter API query.

tweepy.models.ResultSet

In [0]:
# for t in timeline:
#   print(t.in_reply_to_screen_name)

In [0]:
# need the following fields: 
# 1. tweet_id
# 2. authod_id
# 3. inbound
# 4. created_at
# 5. text
# 6. response _tweet_id
# 7. in_response_to_tweet_id


tweet_id = []
authod_id = []
inbound = []
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
for t in timeline: 
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(t.in_reply_to_screen_name != userid)
  created_at.append(t.created_at)
  text.append(t.text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [0]:
for status in tweepy.Cursor(api.user_timeline, screen_name=userid, tweet_mode="extended").items():
    # print(status.full_text)

In [0]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

200
200
200
200
200
200


In [0]:
sgair = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

In [0]:
sgair.shape

(200, 6)

In [0]:
timeline[-1].in_reply_to_status_id_str

'1238325999814631425'

In [0]:
api.get_status('1236079077171355648')._json

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Mar 06 23:59:51 +0000 2020',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/1…',
    'expanded_url': 'https://twitter.com/i/web/status/1236079077171355648',
    'indices': [116, 139],
    'url': 'https://t.co/TKVVH8e8gC'}],
  'user_mentions': [{'id': 253340062,
    'id_str': '253340062',
    'indices': [0, 13],
    'name': 'Singapore Airlines',
    'screen_name': 'SingaporeAir'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1236079077171355648,
 'id_str': '1236079077171355648',
 'in_reply_to_screen_name': 'SingaporeAir',
 'in_reply_to_status_id': 1236078568079319040,
 'in_reply_to_status_id_str': '1236078568079319040',
 'in_reply_to_user_id': 253340062,
 'in_reply_to_user_id_str': '253340062',
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" r

In [0]:
# find the content that the users wrote that sgair is replying to
user_content = {'tweet_id': [],
                'authod_id': [],
                'inbound': [],
                'created_at': [],
                'text': [],
                # 'reponses_tweet_id': [],
                'in_response_to_tweet_id': []}


for t in in_response_to_tweet_id:
  try:
    info = api.get_status(t)._json
    user_content['tweet_id'].append(info['id_str'])
    user_content['authod_id'].append(info['user']['id_str'])
    user_content['inbound'].append(info['user']['id_str'] != info['in_reply_to_status_id_str'])
    user_content['created_at'].append(info['created_at'])
    user_content['text'].append(info['text'])
    user_content['in_response_to_tweet_id'].append(info['in_reply_to_status_id_str'])
  except:
    continue
  

In [0]:
user_content = pd.DataFrame.from_dict(user_content, orient='columns')

In [0]:
user_content.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1241002804321239040,253340062,True,Fri Mar 20 14:04:59 +0000 2020,Update 8: https://t.co/NRNIXuB4cH,1240871409456254976
1,1241467795218370562,47768134,True,Sat Mar 21 20:52:42 +0000 2020,@SingaporeAir @gurknot My travel agent didn’t ...,1241436819142537218
2,1241450306388373506,102592080,True,Sat Mar 21 19:43:12 +0000 2020,@SingaporeAir We are only informed that ticket...,1241312392073613312
3,1241444740291350529,1154586603538378752,True,Sat Mar 21 19:21:05 +0000 2020,@SingaporeAir urgent: I had a reservation for ...,None
4,1241438090540900352,71284049,True,Sat Mar 21 18:54:40 +0000 2020,@SingaporeAir Thank you. Have you accomodated ...,1241435989295239168


In [0]:
sgair.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1241598719872524288,253340062,False,2020-03-22 05:32:57,Update 9: https://t.co/FZV4CSsek5,1241002804321239040
1,1241476348448665600,253340062,True,2020-03-21 21:26:41,"@sanjaypatel03 Hi Sanjay, bookings made via a ...",1241467795218370562
2,1241476109184561152,253340062,True,2020-03-21 21:25:44,"@khanabadossh Hi Ajay, you may wish to provide...",1241450306388373506
3,1241451748457242624,253340062,True,2020-03-21 19:48:56,"@WilliamJGlover Hi William, sorry to hear this...",1241444740291350529
4,1241451410304028672,253340062,True,2020-03-21 19:47:35,"@suri_at_tweet Hi there, please visit https://...",1241438090540900352


In [0]:
all_tweets = pd.concat([sgair, user_content])

In [0]:
all_tweets.shape

(389, 6)

In [0]:
from google.colab import files

# to download the csv
all_tweets.to_csv('sgair_200.csv')
files.download('sgair_200.csv')



---



# AirAsia

## Main Twitter Account -- @AirAsia

In [0]:
userid = 'AirAsia'

In [0]:
# finding out about Singapore Air
airasia_info = api.get_user(screen_name=userid)

In [0]:
# getting Singapore Airlines' 200 most recent posts
timeline = api.user_timeline(screen_name=userid, count=500)

In [0]:
# need the following fields: 
# 1. tweet_id
# 2. authod_id
# 3. inbound
# 4. created_at
# 5. text
# 6. response _tweet_id
# 7. in_response_to_tweet_id


tweet_id = []
authod_id = []
inbound = []
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
# for t in timeline: 
#   tweet_id.append(t.id_str)
#   authod_id.append(t.user.id_str)
#   inbound.append(t.in_reply_to_screen_name != userid)
#   created_at.append(t.created_at)
#   text.append(t.text)
#   # responses_tweet_id.append(t.reply_count)
#   in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [0]:
for t in tweepy.Cursor(api.user_timeline, screen_name=userid, tweet_mode="extended").items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(t.in_reply_to_screen_name != userid)
  created_at.append(t.created_at)
  text.append(t.full_text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [0]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

3245
3245
3245
3245
3245
3245


In [0]:
airasia = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

In [0]:
airasia.shape

(3245, 6)

In [0]:
# find the content that the users wrote that sgair is replying to
user_content = {'tweet_id': [],
                'authod_id': [],
                'inbound': [],
                'created_at': [],
                'text': [],
                # 'reponses_tweet_id': [],
                'in_response_to_tweet_id': []}


for t in in_response_to_tweet_id:
  try:
    info = api.get_status(t)._json
    user_content['tweet_id'].append(info['id_str'])
    user_content['authod_id'].append(info['user']['id_str'])
    user_content['inbound'].append(info['user']['id_str'] != info['in_reply_to_status_id_str'])
    user_content['created_at'].append(info['created_at'])
    user_content['text'].append(info['text'])
    user_content['in_response_to_tweet_id'].append(info['in_reply_to_status_id_str'])
  except:
    continue
  

In [0]:
user_content = pd.DataFrame.from_dict(user_content, orient='columns')

In [0]:
user_content.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1246390088122036224,22919665,True,Sat Apr 04 10:52:08 +0000 2020,Amid the Covid-19 outbreak and Movement Contro...,None
1,1245926754344038401,22919665,True,Fri Apr 03 04:11:01 +0000 2020,A step-by-step guide on how to make full use o...,None
2,1245561840261713920,22919665,True,Thu Apr 02 04:00:58 +0000 2020,(1/2) It just takes 5 to 10 days for you to re...,None
3,1245245530789249024,22919665,True,Wed Apr 01 07:04:04 +0000 2020,We hear you and we're doing more to help you. ...,None
4,1244862054051090432,22919665,True,Tue Mar 31 05:40:16 +0000 2020,(2/3) ... previously paid for your booking. Th...,1244862049617735680


In [0]:
airasia.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1246621692950011909,22919665,True,2020-04-05 02:12:27,#AirAsia Foundation launches digital donation ...,None
1,1246390299565256705,22919665,False,2020-04-04 10:52:58,"This campaign uses our e-commerce platform, OU...",1246390088122036224
2,1246390088122036224,22919665,True,2020-04-04 10:52:08,Amid the Covid-19 outbreak and Movement Contro...,None
3,1245926759465287680,22919665,False,2020-04-03 04:11:02,A step-by-step guide on how to make full use o...,1245926754344038401
4,1245926754344038401,22919665,True,2020-04-03 04:11:01,A step-by-step guide on how to make full use o...,None


In [0]:
main_tweets = pd.concat([airasia, user_content])

In [0]:
main_tweets.shape

(3696, 6)

## Customer Service Twitter Account -- @AirAsiaSupport

In [0]:
userid = 'AirAsiaSupport'

In [0]:
# finding out about Singapore Air
airasiasupp_info = api.get_user(screen_name=userid)

# this returns a User object, params see here: https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object
# for Tweet objects, https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object 

In [0]:
# getting Singapore Airlines' 200 most recent posts
timeline = api.user_timeline(screen_name=userid, count=500)

In [0]:
# need the following fields: 
# 1. tweet_id
# 2. authod_id
# 3. inbound
# 4. created_at
# 5. text
# 6. response _tweet_id
# 7. in_response_to_tweet_id


tweet_id = []
authod_id = []
inbound = []
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
# for t in timeline: 
#   tweet_id.append(t.id_str)
#   authod_id.append(t.user.id_str)
#   inbound.append(t.in_reply_to_screen_name != userid)
#   created_at.append(t.created_at)
#   text.append(t.text)
#   # responses_tweet_id.append(t.reply_count)
#   in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [0]:
for t in tweepy.Cursor(api.user_timeline, screen_name=userid, tweet_mode="extended").items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(t.in_reply_to_screen_name != userid)
  created_at.append(t.created_at)
  text.append(t.full_text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [0]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

3214
3214
3214
3214
3214
3214


In [0]:
airasiasupp = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

In [0]:
airasiasupp.shape

(3214, 6)

In [0]:
# find the content that the users wrote that sgair is replying to
user_content = {'tweet_id': [],
                'authod_id': [],
                'inbound': [],
                'created_at': [],
                'text': [],
                # 'reponses_tweet_id': [],
                'in_response_to_tweet_id': []}


for t in in_response_to_tweet_id:
  try:
    info = api.get_status(t)._json
    user_content['tweet_id'].append(info['id_str'])
    user_content['authod_id'].append(info['user']['id_str'])
    user_content['inbound'].append(info['user']['id_str'] != info['in_reply_to_status_id_str'])
    user_content['created_at'].append(info['created_at'])
    user_content['text'].append(info['text'])
    user_content['in_response_to_tweet_id'].append(info['in_reply_to_status_id_str'])
  except:
    continue
  

In [0]:
user_content = pd.DataFrame.from_dict(user_content, orient='columns')

In [0]:
user_content.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1235421722175627266,769028569,True,Thu Mar 05 04:27:46 +0000 2020,@AirAsiaSupport @AirAsiaSupport I have my book...,None
1,1237522969305305088,1237430368149499904,True,Tue Mar 10 23:37:22 +0000 2020,@AirAsiaSupport Thanks for your reply. Please ...,1237448704149327877
2,1235232214478016514,1024875383970181122,True,Wed Mar 04 15:54:44 +0000 2020,@AirAsiaSupport already DM but no answer.. ver...,1225976008118898688
3,1237627161210580993,1207700967082213376,True,Wed Mar 11 06:31:23 +0000 2020,@AirAsiaSupport can you help me to amend my na...,1225976008118898688
4,1236891148687298560,1008759565,True,Mon Mar 09 05:46:44 +0000 2020,@AirAsiaSupport มีระบบที่เป็นคนจริงๆไหม คุยกับ...,None


In [0]:
airasiasupp.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1246693762782449664,179379726,True,2020-04-05 06:58:50,"@adila_azman Please provide via DM, do not twe...",1243585612181123075
1,1246693591457484800,179379726,True,2020-04-05 06:58:09,"@adila_azman Hi Adila, \n, please provide exa...",1243585612181123075
2,1246687865125339137,179379726,True,2020-04-05 06:35:24,"@KanteWest_ Hello, @KanteWest_. We apologize f...",1244639323988742144
3,1246683557327785985,179379726,True,2020-04-05 06:18:16,"@maj_manjeet Hello, maj_manjeet. We apologize ...",1243220696836091905
4,1246682544294236160,179379726,True,2020-04-05 06:14:15,"@bungamawaarrr Maksudnya, ada ada tak penerban...",1244264850873696262


In [0]:
supp_tweets = pd.concat([airasiasupp, user_content])

In [0]:
supp_tweets.shape

(3230, 6)

In [0]:
# Merge the airasia and airasiasupport together
airasia_tweets = pd.concat([main_tweets, supp_tweets])

In [0]:
from google.colab import files

# to download the csv
airasia_tweets.to_csv('airasia_combined.csv')
files.download('airasia_combined.csv')



---



# Jetstar

In [0]:
userid = 'Jetstar_Asia'

In [0]:

jetasia_info = api.get_user(screen_name=userid)

# this returns a User object, params see here: https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object
# for Tweet objects, https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object 

In [0]:
# need the following fields: 
# 1. tweet_id
# 2. authod_id
# 3. inbound
# 4. created_at
# 5. text
# 6. response _tweet_id
# 7. in_response_to_tweet_id


tweet_id = []
authod_id = []
inbound = []
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
for t in tweepy.Cursor(api.user_timeline, screen_name=userid, tweet_mode="extended").items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(t.in_reply_to_screen_name != userid)
  created_at.append(t.created_at)
  text.append(t.full_text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [0]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

3200
3200
3200
3200
3200
3200


In [0]:
jetasia = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

In [0]:
jetasia.shape

(3200, 6)

In [0]:
# find the content that the users wrote that sgair is replying to
user_content = {'tweet_id': [],
                'authod_id': [],
                'inbound': [],
                'created_at': [],
                'text': [],
                # 'reponses_tweet_id': [],
                'in_response_to_tweet_id': []}


for t in in_response_to_tweet_id:
  try:
    info = api.get_status(t)._json
    user_content['tweet_id'].append(info['id_str'])
    user_content['authod_id'].append(info['user']['id_str'])
    user_content['inbound'].append(info['user']['id_str'] != info['in_reply_to_status_id_str'])
    user_content['created_at'].append(info['created_at'])
    user_content['text'].append(info['text'])
    user_content['in_response_to_tweet_id'].append(info['in_reply_to_status_id_str'])
  except:
    continue
  

In [0]:
user_content = pd.DataFrame.from_dict(user_content, orient='columns')

In [0]:
user_content.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1246635109333127168,84333550,True,Sun Apr 05 03:05:46 +0000 2020,@Jetstar_Asia hello.. i have received a refund...,None
1,1245918910580318208,18406421,True,Fri Apr 03 03:39:50 +0000 2020,"Hi @Jetstar_Asia, i have requested refund on 1...",None
2,1245308857875697664,389351915,True,Wed Apr 01 11:15:43 +0000 2020,Hi @Jetstar_Asia have requested refund on 10 M...,None
3,1245180699247702016,774187325823807488,True,Wed Apr 01 02:46:27 +0000 2020,@Jetstar_Asia your website told me that my jou...,None
4,1245145090491699200,901775758665715713,True,Wed Apr 01 00:24:57 +0000 2020,"@Jetstar_Asia You have $1,700 of my money for ...",None


In [0]:
jetasia.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1246636468396789762,73287339,True,2020-04-05 03:11:10,"@elsadiella Hi Elsa, thanks for your DM. Let u...",1246635109333127168
1,1245925067692707840,73287339,True,2020-04-03 04:04:18,"@emmyoga Hi Emmanuel, our team is working hard...",1245918910580318208
2,1245502987239337988,73287339,True,2020-04-02 00:07:07,"@epulzakaria Hiya, we’d like to look into this...",1245308857875697664
3,1245490068736880642,73287339,True,2020-04-01 23:15:47,"@SzigyartoMarton Hi Marton, can you please sen...",1245239537770659841
4,1245485543229702145,73287339,True,2020-04-01 22:57:48,"@rich_consumer Hi Richard, we’d like to look i...",1245180699247702016


In [0]:
jetasia_all = pd.concat([jetasia, user_content])

In [0]:
jetasia_all.shape

(3970, 6)

# Combining all the datasets

In [0]:
twitter_data = pd.concat([jetasia_all, airasia_tweets])

In [0]:
twitter_data.shape

(10896, 6)

In [0]:
from google.colab import files

# to download the csv
twitter_data.to_csv('twitter_data.csv')
files.download('twitter_data.csv')